### Lab Prog 8 : Implement unification in first order logic

In [17]:
import re
def getAttributes(expr):
    expr = expr.split("(")[1:]
    expr = "(".join(expr)
    expr = expr[:-1]
    expr = re.split("(?<!\(.),(?!.\))", expr)
    return expr

def getInitialPredicate(expression):
    return expression.split("(")[0]


def isConstant(char):
    return char.isupper() and len(char) == 1

def isVariable(char):
    return char.islower() and len(char) == 1

def replaceAttributes(exp, old, new):
    attributes = getAttributes(exp)
    
    for index, val in enumerate(attributes):
        if val == old:
            attributes[index] = new
    predicate = getInitialPredicate(exp)
    return predicate + "(" + ",".join(attributes) + ")"

In [18]:
def apply(exp, subs):
    for sub in subs:
        new, old = sub[0],sub[2]  # sub is a tuple of 2 values (new, old)
        exp = replaceAttributes(exp, old, new)
    return exp

def checkOccurs(var, exp):
    if exp.find(var) == -1:
        return False
    return True


def getFirstPart(exp):
    attributes = getAttributes(exp)
    return attributes[0]


def getRemainingPart(exp):
    predicate = getInitialPredicate(exp)
    attributes = getAttributes(exp)
    newExp = predicate + "(" + ",".join(attributes[1:]) + ")"
    return newExp

In [25]:
def unify(exp1, exp2):
    if exp1 == exp2:
        return []

    if isConstant(exp1) and isConstant(exp2):
        if exp1 != exp2:
            return False

    if isConstant(exp1):
        return [(exp1,'|', exp2)]

    if isConstant(exp2):
        return [(exp2,'|', exp1)]

    if isVariable(exp1):
        if checkOccurs(exp1, exp2):
            
            return False
        else:
            
            return [(exp2,'|', exp1)]

    if isVariable(exp2):
        if checkOccurs(exp2, exp1):
            return False
        else:
            return [(exp1,'|', exp2)]

    if getInitialPredicate(exp1) != getInitialPredicate(exp2):
        print("Predicates do not match. Cannot be unified")
        return False
    
    a1 = len(getAttributes(exp1))
    a2 = len(getAttributes(exp2))
    if a1 != a2:
        return False

    head1 = getFirstPart(exp1)
    print("Head1 ",head1)
    head2 = getFirstPart(exp2)
    print("Head2 ",head2)
    initialSub = unify(head1, head2)
    print("initial ",initialSub)
    
    if not initialSub:
        return False
    if a1 == 1:
        return initialSub

    tail1 = getRemainingPart(exp1)
    print("tail1 ",tail1)
    tail2 = getRemainingPart(exp2)
    print("tail2 ",tail2)

    if initialSub!= []:
        tail1 = apply(tail1, initialSub)
        print("tail11 ",tail1)
        tail2 = apply(tail2, initialSub)
        print("tail21 ",tail2)
        
    remainingSubs= unify(tail1, tail2)
    if not remainingSubs:
        return False
    
    initialSub.extend(remainingSubs)
    return initialSub

In [27]:
#test1
exp1 = "knows(John,y)"
exp2 = "knows(x,Bill)"
subs = unify(exp1, exp2)
print("Substitutions:")
print(subs)

Head1  John
Head2  x
initial  [('John', '|', 'x')]
tail1  knows(y)
tail2  knows(Bill)
tail11  knows(y)
tail21  knows(Bill)
Head1  y
Head2  Bill
initial  [('Bill', '|', 'y')]
Substitutions:
[('John', '|', 'x'), ('Bill', '|', 'y')]


In [28]:
#test2
exp1 = "studies(x,g(x))"
exp2 = "studies(z,f(z))"
subs = unify(exp1, exp2)
print("Substitutions:")
print(subs)

Head1  x
Head2  z
initial  [('z', '|', 'x')]
tail1  studies(g(x))
tail2  studies(f(z))
tail11  studies(g(x))
tail21  studies(f(z))
Head1  g(x)
Head2  f(z)
Predicates do not match. Cannot be unified
initial  False
Substitutions:
False


In [29]:
#test3
exp1 = input("Enter Exp1:  ")
exp2 = input("Enter Exp2:  ")
subs = unify(exp1, exp2)
print("Substitutions:")
print(subs)

Enter Exp1:  motherof(a,b,c)
Enter Exp2:  siblingof(x,y,z)
Predicates do not match. Cannot be unified
Substitutions:
False


In [34]:
#test4
exp1 = input("Enter Exp1:  ")
exp2 = input("Enter Exp2:  ")
subs = unify(exp1, exp2)
print("Substitutions:")
print(subs)

Enter Exp1:  Knows(John,x)
Enter Exp2:  Knows(y,z)
Head1  John
Head2  y
initial  [('John', '|', 'y')]
tail1  Knows(x)
tail2  Knows(z)
tail11  Knows(x)
tail21  Knows(z)
Head1  x
Head2  z
initial  [('z', '|', 'x')]
Substitutions:
[('John', '|', 'y'), ('z', '|', 'x')]
